In [1]:
import requests
import numpy as np
import pandas as pd
#TASK: Make a pie chart on how accesible each station is

In [2]:
#api_url = "https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1"
def db_get(url,params = None):
    api_key = "6d680224d118a0784d34efe88550115e"
    client_id = "d95e9f1c7c8f175d8b787361e851c33f"
    headers = {
    'DB-Client-Id': client_id,
    'DB-Api-Key': api_key,
    'accept': "application/json"
    } 
    
    response = requests.get(url, headers = headers,params = params)
    return response

response = db_get(url = "https://apis.deutschebahn.com/db-api-marketplace/apis/fasta/v2/facilities")
json_file = response.json()
df = pd.DataFrame(json_file)

In [3]:
#print(df.loc[df['stationnumber'] == 626])
#print(df)
#print(json_file)

In [4]:
#Demonstration that some stations dont have facilities
response = db_get(url = "https://apis.deutschebahn.com/db-api-marketplace/apis/fasta/v2/stations/626")
print(response.json())
print(df.loc [df['stationnumber'] == 626])

{'facilities': [], 'name': 'Bienenmühle', 'stationnumber': 626}
Empty DataFrame
Columns: [description, equipmentnumber, geocoordX, geocoordY, operatorname, state, stateExplanation, stationnumber, type]
Index: []


In [5]:
x = df['stationnumber']
print(f"Total number of stations with facilities: {len(set(x.values.tolist()))}")

Total number of stations with facilities: 1271


In [6]:
# Excel file plucked out of the internet to calculate total amount of bahnhofs in germany. (approxiametely 5403 from january)
# data available publicly here: https://www.deutschebahn.com/resource/blob/8889522/1d8184af7653d198d256ce2ce2403fae/Stationspreisliste-2023-data.pdf
df2=pd.read_excel('Station_data.xlsx')
selected = df2[['Bf-Nr','Bahnhof']]
numbertobahn = dict(selected.to_dict('split')['data'])
df['Bahnhof'] = df['stationnumber'].apply(lambda x: numbertobahn[x])

In [7]:
#x= total number of train stations with facilities(functioning w/ non-functioning)

x = len(set(df['stationnumber'].values.tolist()))/len(numbertobahn)
print(f"Percentage of DB train stations with facilities: {x}") #percentage of DB train stations with facilities

Percentage of DB train stations with facilities: 0.23523968165833797


In [8]:
x = df['operatorname'].unique()
print(f"List of facility operators: {x}")

List of facility operators: ['DB Station&Service' 'Stadt Chemnitz' 'Lauffen (Neckar)' 'Stadt Lauda'
 'Eisbrecher_KONE' 'Bad Sobernheim' 'Eisbrecher_Schindler'
 'Stadt Leverkusen' 'Stadt Soest' 'Stadt Werl']


In [10]:
selected = df.loc[:, ~df.columns.isin(['geocoordX', 'geocoordY','description','equipmentnumber','stateExplanation'])]
#Disabled columns for easier overview

a = ['Stadt Chemnitz','Lauffen (Neckar)','Stadt Lauda','Eisbrecher_KONE','Bad Sobernheim','Eisbrecher_Schindler',
     'Stadt Leverkusen','Stadt Soest','Stadt Werl']
#a = selection of Stations
for i in a:
    print(f"{i} Facilities: \n{selected.loc[selected['operatorname'] == i]}")

Stadt Chemnitz Facilities: 
       operatorname   state  stationnumber      type       Bahnhof
631  Stadt Chemnitz  ACTIVE           1040  ELEVATOR  Chemnitz Hbf
633  Stadt Chemnitz  ACTIVE           1040  ELEVATOR  Chemnitz Hbf
Lauffen (Neckar) Facilities: 
         operatorname   state  stationnumber      type           Bahnhof
681  Lauffen (Neckar)  ACTIVE           3591  ELEVATOR  Lauffen (Neckar)
Stadt Lauda Facilities: 
     operatorname   state  stationnumber      type Bahnhof
1551  Stadt Lauda  ACTIVE           3576  ELEVATOR   Lauda
Eisbrecher_KONE Facilities: 
         operatorname     state  stationnumber       type              Bahnhof
1666  Eisbrecher_KONE    ACTIVE            569  ESCALATOR         Berliner Tor
1668  Eisbrecher_KONE    ACTIVE            569  ESCALATOR         Berliner Tor
3009  Eisbrecher_KONE  INACTIVE           6530  ESCALATOR  Wandsbeker Chaussee
Bad Sobernheim Facilities: 
        operatorname   state  stationnumber      type         Bahnhof
1894  Bad

In [11]:
x = df['type'].unique()
print(f"Total types of facilies: {x}")

for i in x:
    print(f"Total number of {i}: {len(selected.loc[selected['type'] == i])}")

Total types of facilies: ['ELEVATOR' 'ESCALATOR']
Total number of ELEVATOR: 2570
Total number of ESCALATOR: 981


In [12]:
#Currently inactive or unknown status facilities
print(f"Total types of status: {df['state'].unique()}")
x = len(df.loc[df['state'] == 'ACTIVE'])
print(f"Total number of ACTIVE status facilities: {x}")
print(f"Total number of INACTIVE/UNKNOWN status facilities: {len(df)-x}")

Total types of status: ['ACTIVE' 'INACTIVE' 'UNKNOWN']
Total number of ACTIVE status facilities: 3127
Total number of INACTIVE/UNKNOWN status facilities: 424
